# Kinship Detection Project

#### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#### Reading csv files

In [2]:
train_df = pd.read_csv("train_relationships.csv")
test_df = pd.read_csv("sample_submission.csv")

In [3]:
train_df.head(10)

,p1,p2
0,F0002/MID1,F0002/MID3
1,F0002/MID2,F0002/MID3
2,F0005/MID1,F0005/MID2
3,F0005/MID3,F0005/MID2
4,F0009/MID1,F0009/MID4
5,F0009/MID1,F0009/MID3
6,F0009/MID1,F0009/MID2
7,F0009/MID1,F0009/MID6
8,F0009/MID2,F0009/MID4
9,F0009/MID2,F0009/MID6


In [4]:
test_df.head(10)

,img_pair,is_related
0,face05508.jpg-face01210.jpg,0
1,face05750.jpg-face00898.jpg,0
2,face05820.jpg-face03938.jpg,0
3,face02104.jpg-face01172.jpg,0
4,face02428.jpg-face05611.jpg,0
5,face01219.jpg-face00274.jpg,0
6,face04262.jpg-face00555.jpg,0
7,face03697.jpg-face01892.jpg,0
8,face03524.jpg-face00319.jpg,0
9,face03410.jpg-face05368.jpg,0


#### Loading pre-trained FaceNet model

In [5]:
from keras.models import load_model

model_path = 'facenet_keras.h5' #Downloaded from 'https://github.com/nyoki-mtl/keras-facenet' 
model = load_model(model_path)

Using TensorFlow backend.









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/nidhi/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


#### Pre-processing images

In [6]:
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def load_and_align_images(filepaths, margin,image_size = 160):
    
    aligned_images = []
    for filepath in filepaths:
        img = imread(filepath)
        aligned = resize(img, (image_size, image_size), mode='reflect')
        aligned_images.append(aligned)
            
    return np.array(aligned_images)

#### Computing Image Embeddings

In [7]:
def calc_embs(filepaths, margin=10, batch_size=512):
    pd = []
    for start in tqdm(range(0, len(filepaths), batch_size)):
        aligned_images = prewhiten(load_and_align_images(filepaths[start:start+batch_size], margin))
        pd.append(model.predict_on_batch(aligned_images))
    embs = l2_normalize(np.concatenate(pd))

    return embs

In [11]:
from imageio import imread
from skimage.transform import resize
from tqdm import tqdm

test_images = os.listdir("test/")
test_embs = calc_embs([os.path.join("test/", f) for f in test_images])
np.save("test_embs.npy", test_embs)

100%|██████████| 13/13 [01:43<00:00,  6.51s/it]
